**Import packages**

In [2]:
import sys
from pathlib import Path

# Get the parent directory of the current notebook
notebook_dir = Path.cwd()  # Folder containing the notebook
project_root = notebook_dir.parent  # Go up one level to 'project/'

# Add the 'src/' folder to Python's search path
sys.path.append(str(project_root / "src"))
sys.path.append(str(project_root / "data"))

# Verify
print(sys.path)
import yfinance as yf
from fredapi import Fred
import pandas as pd 
from dotenv import load_dotenv
import os
from ecbdata import ecbdata
import prophet as Prophet
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error

import numpy as np
from sklearn.preprocessing import StandardScaler
from prophet import Prophet
import plotly as plt 
import itertools


from dask.distributed import Client
from prophet.diagnostics import cross_validation , performance_metrics

['C:\\Users\\Radhouen\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\Radhouen\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\Radhouen\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\Radhouen\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\venv', '', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\venv\\lib\\site-packages', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\venv\\lib\\site-packages\\win32', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\src', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\data']


c:\Users\Radhouen\Gold_Diggers\gold-team\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Load Dataframe**

In [3]:
df=pd.read_csv('df_not_stationary.csv')
df=df.set_index(df['Date'])
df.head(30)

,Date,Gold,USD_Index,EUR_USD,SP_500,Oil,Bitcoin,Vix,CPI,Unemployment_Rate,...,Zinsen,Gold_ETFs,M2_Euro,M2_US,GPRD,DOW,MOY,Stunde_des_Tages,Jahr,Volles_Datum
Date,,,,,,,,,,,,,,,,,,,,,
2014-09-17,2014-09-17,1234.400024,84.699997,1.295908,2001.569946,94.419998,457.334015,12.650000,237.712067,5.793333,...,0.09,17.140000,9.487253e+12,11543.060000,159.797913,2,9,0,2014,2014-09-17
2014-09-18,2014-09-18,1225.699951,84.320000,1.285000,2011.359985,93.070000,424.440002,12.030000,237.692133,5.786667,...,0.09,15.500000,9.488013e+12,11545.520000,118.057968,3,9,0,2014,2014-09-18
2014-09-19,2014-09-19,1215.300049,84.800003,1.292006,2010.400024,92.410004,394.795990,12.110000,237.672200,5.780000,...,0.09,16.900000,9.488774e+12,11547.980000,132.806259,4,9,0,2014,2014-09-19
2014-09-20,2014-09-20,1215.300049,84.800003,1.292006,2010.400024,92.410004,408.903992,12.110000,237.652267,5.773333,...,0.09,16.746667,9.489535e+12,11550.440000,76.326447,5,9,0,2014,2014-09-20
2014-09-21,2014-09-21,1215.300049,84.800003,1.292006,2010.400024,92.410004,398.821014,12.110000,237.632333,5.766667,...,0.09,16.593333,9.490296e+12,11552.900000,79.820908,6,9,0,2014,2014-09-21
2014-09-22,2014-09-22,1216.800049,84.669998,1.284142,1994.290039,91.519997,402.152008,13.690000,237.612400,5.760000,...,0.09,16.440000,9.491056e+12,11555.360000,80.678589,0,9,0,2014,2014-09-22
2014-09-23,2014-09-23,1221.000000,84.660004,1.285149,1982.770020,91.559998,435.790985,14.930000,237.592467,5.753333,...,0.09,16.650000,9.491817e+12,11557.820000,179.175827,1,9,0,2014,2014-09-23
2014-09-24,2014-09-24,1218.599976,85.050003,1.285397,1998.300049,92.800003,423.204987,13.270000,237.572533,5.746667,...,0.09,17.210000,9.492578e+12,11560.280000,146.881638,2,9,0,2014,2014-09-24
2014-09-25,2014-09-25,1221.199951,85.199997,1.277808,1965.989990,92.529999,411.574005,15.640000,237.552600,5.740000,...,0.09,17.990000,9.493339e+12,11562.740000,107.528381,3,9,0,2014,2014-09-25


**Preparing the dataframe for Prophet**

In [4]:

gold_df=df[['Gold','GPD','M2_US','CPI','SP_500','Zinsen','Bitcoin']].reset_index()
gold_df=gold_df.rename(columns={'Gold': 'y','Date':'ds'})
gold_df


,ds,y,GPD,M2_US,CPI,SP_500,Zinsen,Bitcoin
0,2014-09-17,1234.400024,17895.666891,11543.06,237.712067,2001.569946,0.09,457.334015
1,2014-09-18,1225.699951,17896.839185,11545.52,237.692133,2011.359985,0.09,424.440002
2,2014-09-19,1215.300049,17898.011478,11547.98,237.672200,2010.400024,0.09,394.795990
3,2014-09-20,1215.300049,17899.183772,11550.44,237.652267,2010.400024,0.09,408.903992
4,2014-09-21,1215.300049,17900.356065,11552.90,237.632333,2010.400024,0.09,398.821014
...,...,...,...,...,...,...,...,...
3911,2025-06-02,3370.600098,29976.638000,21862.50,320.795000,5935.939941,4.33,105881.531250
3912,2025-06-03,3350.199951,29976.638000,21862.50,320.795000,5970.370117,4.33,105432.468750
3913,2025-06-04,3373.500000,29976.638000,21862.50,320.795000,5970.810059,4.33,104731.984375
3914,2025-06-05,3350.699951,29976.638000,21862.50,320.795000,5939.299805,4.33,101575.953125


**Filling NA values, if they exist**

In [5]:
# 3. Handle missing data
gold_df.fillna(method='ffill', inplace=True)
gold_df.fillna(method='bfill', inplace=True)

C:\Users\Radhouen\AppData\Local\Temp\ipykernel_13924\4024275292.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gold_df.fillna(method='ffill', inplace=True)
C:\Users\Radhouen\AppData\Local\Temp\ipykernel_13924\4024275292.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gold_df.fillna(method='bfill', inplace=True)


In [6]:
gold_df

,ds,y,GPD,M2_US,CPI,SP_500,Zinsen,Bitcoin
0,2014-09-17,1234.400024,17895.666891,11543.06,237.712067,2001.569946,0.09,457.334015
1,2014-09-18,1225.699951,17896.839185,11545.52,237.692133,2011.359985,0.09,424.440002
2,2014-09-19,1215.300049,17898.011478,11547.98,237.672200,2010.400024,0.09,394.795990
3,2014-09-20,1215.300049,17899.183772,11550.44,237.652267,2010.400024,0.09,408.903992
4,2014-09-21,1215.300049,17900.356065,11552.90,237.632333,2010.400024,0.09,398.821014
...,...,...,...,...,...,...,...,...
3911,2025-06-02,3370.600098,29976.638000,21862.50,320.795000,5935.939941,4.33,105881.531250
3912,2025-06-03,3350.199951,29976.638000,21862.50,320.795000,5970.370117,4.33,105432.468750
3913,2025-06-04,3373.500000,29976.638000,21862.50,320.795000,5970.810059,4.33,104731.984375
3914,2025-06-05,3350.699951,29976.638000,21862.50,320.795000,5939.299805,4.33,101575.953125


**Preparing the regressors**

In [7]:
regressors = ['GPD', 'M2_US', 'CPI', 'SP_500', 'Zinsen', 'Bitcoin']


**Data Splitting**

In [8]:
train = gold_df[(gold_df['ds'] >= '2024-01-01') & (gold_df['ds']<'2025-05-01')]
test = gold_df[gold_df['ds'] >= '2025-05-01']
train

,ds,y,GPD,M2_US,CPI,SP_500,Zinsen,Bitcoin
3393,2024-01-01,2062.399902,28624.069000,20773.200000,308.417000,4769.830078,5.33,44167.332031
3394,2024-01-02,2064.399902,28628.383780,20776.038710,308.478581,4742.830078,5.33,44957.968750
3395,2024-01-03,2034.199951,28632.698560,20778.877419,308.540161,4704.810059,5.33,42848.175781
3396,2024-01-04,2042.300049,28637.013341,20781.716129,308.601742,4688.680176,5.33,44179.921875
3397,2024-01-05,2042.400024,28641.328121,20784.554839,308.663323,4697.240234,5.33,44162.691406
...,...,...,...,...,...,...,...,...
3874,2025-04-26,3282.399902,29976.638000,21862.500000,320.795000,5525.209961,4.33,94646.929688
3875,2025-04-27,3282.399902,29976.638000,21862.500000,320.795000,5525.209961,4.33,93754.843750
3876,2025-04-28,3332.500000,29976.638000,21862.500000,320.795000,5528.750000,4.33,94978.750000
3877,2025-04-29,3318.800049,29976.638000,21862.500000,320.795000,5560.830078,4.33,94284.789062


**Cross-Validation and finding the best parameters**

In [9]:



client = Client()  # connect to the cluster
cutoffs = pd.to_datetime(['2024-02-15', '2024-08-15','2025-02-15'])


param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.3, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 5, 7, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params) 
    for reg in regressors: 
        m.add_regressor(reg)
    m.fit(train) # Fit model with given params
    df_cv = cross_validation(m, initial='100 days', horizon='30 days', parallel="dask")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

11:32:28 - cmdstanpy - INFO - Chain [1] start processing
11:32:29 - cmdstanpy - INFO - Chain [1] done processing
11:32:35 - cmdstanpy - INFO - Chain [1] start processing
11:32:35 - cmdstanpy - INFO - Chain [1] done processing
11:32:38 - cmdstanpy - INFO - Chain [1] start processing
11:32:38 - cmdstanpy - INFO - Chain [1] done processing
11:32:40 - cmdstanpy - INFO - Chain [1] start processing
11:32:40 - cmdstanpy - INFO - Chain [1] done processing
11:32:43 - cmdstanpy - INFO - Chain [1] start processing
11:32:43 - cmdstanpy - INFO - Chain [1] done processing
11:32:46 - cmdstanpy - INFO - Chain [1] start processing
11:32:46 - cmdstanpy - INFO - Chain [1] done processing
11:32:49 - cmdstanpy - INFO - Chain [1] start processing
11:32:49 - cmdstanpy - INFO - Chain [1] done processing
11:32:52 - cmdstanpy - INFO - Chain [1] start processing
11:32:52 - cmdstanpy - INFO - Chain [1] done processing
11:32:55 - cmdstanpy - INFO - Chain [1] start processing
11:32:55 - cmdstanpy - INFO - Chain [1]

    changepoint_prior_scale  seasonality_prior_scale        rmse
0                     0.001                     0.01  241.370548
1                     0.001                     0.10  242.057334
2                     0.001                     1.00  293.281632
3                     0.001                     5.00  289.077176
4                     0.001                     7.00  282.743224
5                     0.001                    10.00  286.955133
6                     0.010                     0.01  152.780489
7                     0.010                     0.10  154.052139
8                     0.010                     1.00  155.748914
9                     0.010                     5.00  154.428538
10                    0.010                     7.00  154.247172
11                    0.010                    10.00  154.718865
12                    0.100                     0.01  150.774052
13                    0.100                     0.10  160.826511
14                    0.1

**Best Parameters Dataframe**

In [10]:
tuning_results.sort_values(by='rmse')

,changepoint_prior_scale,seasonality_prior_scale,rmse
12,0.100,0.01,150.774052
6,0.010,0.01,152.780489
7,0.010,0.10,154.052139
10,0.010,7.00,154.247172
16,0.100,7.00,154.298643
9,0.010,5.00,154.428538
11,0.010,10.00,154.718865
17,0.100,10.00,155.354368
8,0.010,1.00,155.748914
15,0.100,5.00,157.470874


**Training the model with the best parameters**

In [11]:
model = Prophet(
    changepoint_prior_scale=0.1,  
    seasonality_prior_scale=0.01,  
    n_changepoints=50,
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    growth='linear'
)

for reg in regressors :
    model.add_regressor(reg)



model.fit(train.rename(columns={'y': 'y'}))

11:34:19 - cmdstanpy - INFO - Chain [1] start processing
11:34:19 - cmdstanpy - INFO - Chain [1] done processing


**Prediction**

In [12]:
import plotly_express as px 
test_future = test[['ds'] + regressors ].copy()
forecast = model.predict(test_future)
forecast


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Bitcoin,Bitcoin_lower,Bitcoin_upper,CPI,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2025-05-01,7443.490762,3273.605466,3347.976314,7443.490762,7443.490762,0.005606,0.005606,0.005606,-0.052957,...,0.001445,0.001445,0.001445,0.005294,0.005294,0.005294,0.0,0.0,0.0,3313.881656
1,2025-05-02,7460.321689,3266.782443,3343.882327,7460.321689,7460.328611,0.005703,0.005703,0.005703,-0.052957,...,0.000552,0.000552,0.000552,0.002412,0.002412,0.002412,0.0,0.0,0.0,3306.913072
2,2025-05-03,7477.152615,3251.640413,3323.401749,7477.052337,7477.291833,0.005467,0.005467,0.005467,-0.052957,...,-0.000157,-0.000157,-0.000157,-0.000493,-0.000493,-0.000493,0.0,0.0,0.0,3285.578900
3,2025-05-04,7493.983542,3227.586836,3299.960505,7493.648317,7494.292413,0.005101,0.005101,0.005101,-0.052957,...,-0.000734,-0.000734,-0.000734,-0.003374,-0.003374,-0.003374,0.0,0.0,0.0,3264.323758
4,2025-05-05,7510.814468,3207.785939,3279.087151,7510.240252,7511.441018,0.005201,0.005201,0.005201,-0.052957,...,-0.000897,-0.000897,-0.000897,-0.006185,-0.006185,-0.006185,0.0,0.0,0.0,3244.334395
5,2025-05-06,7527.645395,3197.666668,3272.213674,7526.691880,7528.693197,0.005678,0.005678,0.005678,-0.052957,...,0.000073,0.000073,0.000073,-0.008879,-0.008879,-0.008879,0.0,0.0,0.0,3235.306724
6,2025-05-07,7544.476321,3190.808526,3262.202453,7543.078207,7545.940681,0.005731,0.005731,0.005731,-0.052957,...,-0.000282,-0.000282,-0.000282,-0.011417,-0.011417,-0.011417,0.0,0.0,0.0,3224.994697
7,2025-05-08,7561.307248,3209.666646,3278.694619,7559.543713,7563.301955,0.007170,0.007170,0.007170,-0.052957,...,0.001445,0.001445,0.001445,-0.013760,-0.013760,-0.013760,0.0,0.0,0.0,3243.615489
8,2025-05-09,7578.138174,3189.462059,3261.669914,7575.820621,7580.858708,0.007107,0.007107,0.007107,-0.052957,...,0.000552,0.000552,0.000552,-0.015877,-0.015877,-0.015877,0.0,0.0,0.0,3226.911201
9,2025-05-10,7594.969101,3182.436563,3255.133064,7592.133876,7598.459105,0.007507,0.007507,0.007507,-0.052957,...,-0.000157,-0.000157,-0.000157,-0.017742,-0.017742,-0.017742,0.0,0.0,0.0,3217.564068


**Visualization of the prediction and actual value of the price of gold for the last week of the test dataframe**

In [13]:
import plotly_express as px
viz_df = test.copy()
forecasted_values = forecast.tail(len(test))['yhat']
viz_df['yhat'] = forecasted_values.values
px.line(viz_df,viz_df.ds,[viz_df.y,viz_df.yhat])

**RMSE calculated the last week**

In [ ]:

root_mean_squared_error(viz_df.y, viz_df.yhat)

74.56956959485393

**MAPE calculated for the last week**

In [15]:
mean_absolute_percentage_error(viz_df.y, viz_df.yhat)


0.01883564662905551

In [16]:
viz_df.tail(20)

,ds,y,GPD,M2_US,CPI,SP_500,Zinsen,Bitcoin,yhat
3896,2025-05-18,3182.000000,29976.638,21862.5,320.795,5958.379883,4.33,106446.007812,3281.872572
3897,2025-05-19,3228.899902,29976.638,21862.5,320.795,5963.600098,4.33,105606.179688,3289.761268
3898,2025-05-20,3280.300049,29976.638,21862.5,320.795,5940.459961,4.33,106791.085938,3306.295390
3899,2025-05-21,3309.300049,29976.638,21862.5,320.795,5844.609863,4.33,109678.078125,3304.208502
3900,2025-05-22,3292.300049,29976.638,21862.5,320.795,5842.009766,4.33,111673.281250,3332.142711
3901,2025-05-23,3363.600098,29976.638,21862.5,320.795,5802.819824,4.33,107287.796875,3321.859084
3902,2025-05-24,3363.600098,29976.638,21862.5,320.795,5802.819824,4.33,107791.156250,3327.639344
3903,2025-05-25,3363.600098,29976.638,21862.5,320.795,5802.819824,4.33,109035.390625,3334.811073
3904,2025-05-26,3363.600098,29976.638,21862.5,320.795,5802.819824,4.33,109440.367188,3342.381040
3905,2025-05-27,3299.100098,29976.638,21862.5,320.795,5921.540039,4.33,108994.640625,3375.409132


**Pre-Submission**

In [17]:
future_dates = pd.DataFrame({
    'ds': pd.date_range(start='2025-06-09', end='2025-06-13')
})

for reg in regressors:
    future_dates[reg] = gold_df[reg].iloc[-1]  


last_gold_price = gold_df['y'].iloc[-1]




forecast = model.predict(future_dates)

predictions = forecast[['ds', 'yhat']].rename(columns={
    'ds': 'date',
    'yhat': 'prediction'
})

predictions['prediction']=predictions['prediction'].round(2)
predictions['date'] = predictions['date'].dt.strftime('%Y-%m-%d')

predictions.to_csv('pre_submission2.csv', index=False)

print("Predictions for June 9-13, 2025:")
print(predictions)

Predictions for June 9-13, 2025:
         date  prediction
0  2025-06-09     3256.93
1  2025-06-10     3248.76
2  2025-06-11     3230.52
3  2025-06-12     3230.39
4  2025-06-13     3210.54


In [18]:
predictions=pd.read_csv('pre_submission2.csv')
predictions['actual']=[3432.00,3388.36,3346.88,3328.19,3326.89]
print(root_mean_squared_error(predictions['actual'],predictions['prediction']))

131.7417754548648


In [19]:
px.line(predictions, x=predictions.date,y=[predictions.actual,predictions.prediction])

**Prediction starting from 9.6 until 22.6**

In [28]:
future_dates = pd.DataFrame({
    'ds': pd.date_range(start='2025-06-09', end='2025-06-22')
})

for reg in regressors:
    future_dates[reg] = gold_df[reg].iloc[-1]  


last_gold_price = gold_df['y'].iloc[-1]




forecast = model.predict(future_dates)

predictions = forecast[['ds', 'yhat']].rename(columns={
    'ds': 'date',
    'yhat': 'prediction'
})

predictions['prediction']=predictions['prediction'].round(2)
predictions['date'] = predictions['date'].dt.strftime('%Y-%m-%d')



predictions=predictions.drop([5,6,12,13])
predictions['Actual']=[3326.89,3328.18,3346.87,3388.35,3432.00,3383.79,3385.58,3366.77,3371.25,3365.18]
predictions

,date,prediction,Actual
0,2025-06-09,3256.93,3326.89
1,2025-06-10,3248.76,3328.18
2,2025-06-11,3230.52,3346.87
3,2025-06-12,3230.39,3388.35
4,2025-06-13,3210.54,3432.00
7,2025-06-16,3174.77,3383.79
8,2025-06-17,3180.58,3385.58
9,2025-06-18,3178.86,3366.77
10,2025-06-19,3197.88,3371.25
11,2025-06-20,3198.89,3365.18


**RMSE calculated starting from 9.6 until 22.6**

In [29]:
print(root_mean_squared_error(predictions.Actual,predictions.prediction))

166.57406857011082
